# 📚 Booksmith - Book Generation

Simple interface for generating complete books using AI.


## Setup


In [ ]:
import sys
import os
import pickle
import gzip
from pathlib import Path

# Add parent directory to path
sys.path.append(os.path.dirname(os.getcwd()))

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Import Booksmith modules
from booksmith import (
    Book, Character, Chapter,
    LLMConfig,
    WritingAgent
)

# Import EPUB generator
from booksmith.utils.epub_generator import create_book_epub

print("✅ Modules imported successfully")


## 1. Choose Backend and Initialize Agent


In [ ]:
# Configure OpenAI backend
config = LLMConfig(
    model_name="gpt-4.1", 
    max_tokens=32768, 
    temperature=0.7, 
    api_key=os.environ.get("OPENAI_API_KEY")
)

# Initialize agent
agent = WritingAgent(config)

print(f"✅ Agent initialized with OpenAI backend")
print(f"📦 Model: {config.model_name}")
print(f"💾 Memory usage: {agent.get_memory_usage()}")


## 2. Initialize Book Object


In [ ]:
# Create book with your prompt and preferences
book = Book(
    base_prompt="A Clara Costa é enfermeira nos cuidados intensivos num hospital e o João Caetano é Engenheiro de IA. A Clara é elegante e magra, com um cabelo e uns olhos lindos castanhos. O João é mais bem constituido, tem cabelo e olhos castanhos escuros. Eles estão noivos e moram juntos num pequeno apartamento em Moscavide. Gostam muito de ir a Tavira, é o happy place deles. Os dois iam para lá com as respectivas familias desde pequenos sem nunca se teram encontrado lá. Os dois são o maximo felizes quando lá estão juntos. O João gosta também muito de hikes e contacto com a natureza mas a Clara perfere ficar mais por casa ou ir à praia. Os dois são muito amigos estando sempre na galhofa. A historia é passada maioritariamente em Tavira",
    genre="Literary Fiction",
    writing_style="Sophia de Mello Breyner Andresen writing style",
    target_audience="Children and Young Readers",
    language="portuguese from Portugal"
)

print("📚 Book initialized")
print(f"Genre: {book.genre}")
print(f"Style: {book.writing_style}")
print(f"Language: {book.language}")


## 3. Generate Story Summary


In [ ]:
# Generate story summary
agent.generate_story_summary(book)

print("✅ Story summary generated")
print(f"📝 Summary ({len(book.story_summary)} chars):")
print("-" * 50)
print(book.story_summary)
print("-" * 50)


## 4. Generate Book Title


In [ ]:
# Generate book title
agent.generate_title(book)

print("✅ Book title generated")
print(f"📚 Title: \"{book.title}\"")


## 5. Generate Characters


In [ ]:
# Generate characters
agent.generate_characters(book)

print(f"✅ Characters generated ({len(book.characters)} characters)")
for i, character in enumerate(book.characters, 1):
    print(f"\n{i}. {character.name}")
    print(f"   Role: {character.role}")
    print(f"   Personality: {character.personality}")
    if character.appearance:
        print(f"   Appearance: {character.appearance}")
    if character.background_story:
        print(f"   Background: {character.background_story}")


## 6. Generate Chapter Plan


In [ ]:
# Generate chapter plan
agent.generate_chapter_plan(book)

print(f"✅ Chapter plan generated ({len(book.chapters)} chapters)")
for chapter in book.chapters:
    print(f"\nChapter {chapter.chapter_number}: {chapter.title}")
    print(f"Summary: {chapter.summary}")
    if chapter.key_characters:
        print(f"Key characters: {', '.join(chapter.key_characters)}")
    if chapter.plot_points:
        print(f"Plot points: {'; '.join(chapter.plot_points)}")


## 7. Generate All Chapter Content


In [ ]:
# Generate content for all chapters
total_words = 0
for i, chapter in enumerate(book.chapters, 1):
    print(f"📝 Writing Chapter {chapter.chapter_number}: {chapter.title}")
    agent.write_chapter_content(book, chapter)
    
    word_count = len(chapter.content.split()) if chapter.content else 0
    total_words += word_count
    print(f"   ✅ {word_count:,} words written")
    
    # Show preview of first chapter
    if i == 1 and chapter.content:
        preview = chapter.content[:200] + "..." if len(chapter.content) > 200 else chapter.content
        print(f"   Preview: {preview}")

print(f"\n✅ All chapters written")
print(f"📊 Total words: {total_words:,}")
print(f"📊 Average per chapter: {total_words // len(book.chapters):,}")


## 8. Optional: Complete Book Generation (Alternative)


In [ ]:
# Alternative: Use the complete book generation in one call
# Uncomment the lines below to generate everything at once instead of step-by-step

# new_book = Book(
#     base_prompt="Your story prompt here...",
#     genre="Your genre",
#     writing_style="Your preferred style",
#     target_audience="Your audience",
#     language="Your language"
# )
# 
# print("🚀 Generating complete book...")
# agent.write_full_book(new_book)
# print("✅ Complete book generated!")

print("💡 This section shows how to generate a complete book in one function call.")
print("💡 Uncomment the code above to use this alternative approach.")


## 9. Save Book and Generate EPUB


In [ ]:
# Create book-specific folder and save files
from datetime import datetime

# Create book folder name using title and timestamp
book_title_safe = "".join(c for c in book.title if c.isalnum() or c in (' ', '-', '_')).rstrip()
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
book_folder_name = f"{book_title_safe}_{timestamp}"
book_folder_path = Path("notebooks") / book_folder_name

# Create the folder
book_folder_path.mkdir(parents=True, exist_ok=True)

# Save as pickle file
pickle_filename = f"{book_title_safe}.pkl.gz"
pickle_path = book_folder_path / pickle_filename

with gzip.open(pickle_path, 'wb') as f:
    pickle.dump(book, f)

# Generate EPUB using the epub_generator
epub_folder = create_book_epub(book, str(book_folder_path))

print(f"✅ Book saved successfully!")
print(f"📁 Folder: {book_folder_path}")
print(f"💾 Pickle file: {pickle_filename}")
print(f"📚 EPUB generated in folder")

# Display final stats
total_words = sum(len(ch.content.split()) for ch in book.chapters if ch.content)
print(f"\n📊 Final Book Statistics:")
print(f"Title: {book.title}")
print(f"Chapters: {len(book.chapters)}")
print(f"Characters: {len(book.characters)}")
print(f"Total words: {total_words:,}")


In [ ]:
# read a pickled book
import pickle
import gzip
from pathlib import Path

pickle_path = Path("/Users/joaocaetano/projects/Booksmith/notebooks/notebooks/Leonor e a Concha das Vozes Perdidas_20250722_162200/Leonor e a Concha das Vozes Perdidas.pkl.gz")
with gzip.open(pickle_path, 'rb') as f:
    book = pickle.load(f)

epub_folder = create_book_epub(book, str("/Users/joaocaetano/projects/Booksmith/notebooks/notebooks/Leonor e a Concha das Vozes Perdidas_20250722_162200"))